Accuratezza media nel rilevamento delle aree/stanze dell’edificio

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Carica il dataset delle stanze
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.fillna(0, inplace=True)  # Gestisci i valori mancanti
    return df


In [2]:
# Prepara i dati per il modello
def prepare_data(df):
    le = LabelEncoder()
    df['room'] = le.fit_transform(df['room'])  # Codifica le stanze con numeri interi
    X = df.drop(columns=['room'])  # Caratteristiche (RSSI degli AP)
    y = df['room']  # Target (stanze)
    return X, y, le


In [3]:
from sklearn.ensemble import RandomForestClassifier

def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model


In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_predictions(y_test, y_pred, le):
    # Calcola l'accuratezza
    accuracy = accuracy_score(y_test, y_pred)
    
    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    
    return accuracy


In [5]:
'''def analyze_difficult_rooms(y_test, y_pred, le):
    errors = [(true, pred) for true, pred in zip(y_test, y_pred) if true != pred]
    difficult_rooms = pd.DataFrame(errors, columns=['True Room', 'Predicted Room'])
    difficult_counts = difficult_rooms.value_counts()
    print("\nStanze difficili da prevedere:")
    print(difficult_counts)'''
def analyze_difficult_rooms(y_test, y_pred, le):
    # Inverte le etichette numeriche per ottenere i nomi delle stanze
    true_rooms = le.inverse_transform(y_test)
    predicted_rooms = le.inverse_transform(y_pred)
    
    # Rimuovi \n o altri caratteri indesiderati dai nomi delle stanze
    true_rooms = [room.replace("\n", "").strip() for room in true_rooms]
    predicted_rooms = [room.replace("\n", "").strip() for room in predicted_rooms]
    
    # Crea un DataFrame con i risultati
    errors = [(true, pred) for true, pred in zip(true_rooms, predicted_rooms) if true != pred]
    difficult_rooms = pd.DataFrame(errors, columns=['True Room', 'Predicted Room'])
    
    # Conta le stanze difficili da prevedere
    difficult_counts = difficult_rooms.value_counts()
    
    print("\nStanze difficili da prevedere:")
    print(difficult_counts)



In [6]:
from sklearn.metrics import mean_absolute_error

def calculate_position_error(true_positions, predicted_positions):
    errors = [mean_absolute_error(true, pred) for true, pred in zip(true_positions, predicted_positions)]
    avg_error = sum(errors) / len(errors)
    print(f"Errore medio sulla posizione stimata: {avg_error:.2f}")


In [7]:
import logging

logging.basicConfig(level=logging.INFO)

def log_model_training(model):
    try:
        logging.info("Modello addestrato con successo.")
    except Exception as e:
        logging.error(f"Errore durante l'addestramento del modello: {e}")

def log_predictions(y_test, y_pred):
    try:
        accuracy = accuracy_score(y_test, y_pred)
        logging.info(f"Accuratezza della predizione: {accuracy:.2f}")
    except Exception as e:
        logging.error(f"Errore durante la valutazione delle predizioni: {e}")


Validation Accuracy: 0.94
Test Accuracy: 0.85


Classification Report:
              precision    recall  f1-score   support

Dipartimento       1.00      0.67      0.80         3
          E1       1.00      1.00      1.00         1
          E2       0.80      1.00      0.89         4
      Garden       1.00      1.00      1.00         6
 Magazzino1
       0.64      1.00      0.78         7
  Magazzino2       1.00      0.20      0.33         5
  Magazzino3       1.00      1.00      1.00         4
      Uffici       1.00      1.00      1.00         4

    accuracy                           0.85        34
   macro avg       0.93      0.86      0.85        34
weighted avg       0.90      0.85      0.83        34

Accuratezza media nel rilevamento delle stanze: 0.85

Stanze difficili da prevedere:
True Room     Predicted Room
Magazzino2    Magazzino1        4
Dipartimento  E2                1
Name: count, dtype: int64


In [8]:
file_path = r"..\backend\wifi_data_with_rooms.csv"  # Sostituisci con il percorso corretto del file
df = load_data(file_path)
X, y_encoded, le = prepare_data(df)

X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = train_model(X_train, y_train)

# Valuta sui set di validazione e test
val_accuracy = model.score(X_val, y_val)
test_accuracy = model.score(X_test, y_test)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

y_pred = model.predict(X_test)

accuracy = evaluate_predictions(y_test, y_pred, le)
print(f"Accuratezza media nel rilevamento delle stanze: {accuracy:.2f}")

# Analisi delle stanze difficili
analyze_difficult_rooms(y_test, y_pred, le)
